# VacationPy

In [1]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import Google API key
from api_keys import g_key

In [2]:
# Load WeatherPy CSV to DataFrame
cities_data = pd.read_csv("cities.csv", index_col=0)

### Humidity heatmap

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Use latidude and longitude as locations and humidity as weight
city_locations = cities_data[["Latitude", "Longitude"]]
weights = cities_data[["Humidity (%)"]].astype(float)

In [15]:
# Create map and add heat layer to it
fig = gmaps.figure(zoom_level=3, center=(41.9, 87.7))

heat_layer = gmaps.heatmap_layer(
    city_locations,  
    weights=weights['Humidity (%)'],
    dissipating=False,
    max_intensity=100,
    point_radius = 1,
)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [16]:
# Make sure we have ten or fewer rows
destinations = cities_data[cities_data['Max. temperature (F)'] < 80]
destinations = destinations[destinations['Max. temperature (F)'] > 70]
destinations = destinations[destinations['Wind speed (mph)'] < 10]
destinations = destinations[destinations['Cloud coverage (%)'] == 0]

# Add "Hotel name" column to DataFrame
destinations['Hotel name'] = ""
destinations

,City,Country,Latitude,Longitude,Datetime,Max. temperature (F),Humidity (%),Cloud coverage (%),Wind speed (mph),Hotel name
8,Matara,LK,5.95,80.54,2020-02-11 01:04:13,72.90,85,0,9.28,
42,Namibe,AO,-15.20,12.15,2020-02-11 01:04:16,73.67,76,0,7.74,
85,Grand River South East,LK,5.97,80.42,2020-02-11 01:05:21,72.90,85,0,9.28,
129,Kargil,LK,6.12,81.12,2020-02-11 01:06:25,75.20,100,0,6.93,
149,Lagoa,YT,-12.79,45.27,2020-02-11 01:06:27,78.80,94,0,3.36,
179,Dunedin,TD,8.27,16.94,2020-02-11 01:07:30,73.13,20,0,5.28,
430,Kruisfontein,IN,10.57,72.64,2020-02-11 01:12:57,79.97,74,0,7.07,
482,Sao Miguel Do Iguacu,AU,-35.12,139.27,2020-02-11 01:13:03,79.00,86,0,5.01,


### Hotel map

In [8]:
# Hit the Google Places API for each city's coordinates
# Find closest hotel to coordinates
# Assemble url and make API request, store first hotel result into DataFrame

hotels = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    }

In [9]:
for index, row in destinations.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat},{lng}"

    try:
        response = requests.get(base_url, params=params)
        results = response.json()
        destinations.loc[index, "Hotel name"] = results['results'][0]['name']
        hotels.append(results['results'][0]['name'])
        print(f"Closest lodging is {results['results'][0]['name']}.")
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
print("Done")        

Closest lodging is Amaloh Boutique Resort.
Closest lodging is Chik-Chik Namibe.
Closest lodging is Weligama Bay Resort.
Closest lodging is Bungalow 63.
Closest lodging is Le Rocher.
Missing field/result... skipping.
Closest lodging is Hotels in Lakshadweep Islands.
Closest lodging is Balcony On Sixth Lodge.
Done


In [10]:
# Remove rows with NaN and/or empty strings
nan_value = float("NaN")
destinations.replace("", nan_value, inplace=True)
destinations.dropna(subset = ["Hotel name"], inplace=True)
destinations

,City,Country,Latitude,Longitude,Datetime,Max. temperature (F),Humidity (%),Cloud coverage (%),Wind speed (mph),Hotel name
8,Matara,LK,5.95,80.54,2020-02-11 01:04:13,72.90,85,0,9.28,Amaloh Boutique Resort
42,Namibe,AO,-15.20,12.15,2020-02-11 01:04:16,73.67,76,0,7.74,Chik-Chik Namibe
85,Grand River South East,LK,5.97,80.42,2020-02-11 01:05:21,72.90,85,0,9.28,Weligama Bay Resort
129,Kargil,LK,6.12,81.12,2020-02-11 01:06:25,75.20,100,0,6.93,Bungalow 63
149,Lagoa,YT,-12.79,45.27,2020-02-11 01:06:27,78.80,94,0,3.36,Le Rocher
430,Kruisfontein,IN,10.57,72.64,2020-02-11 01:12:57,79.97,74,0,7.07,Hotels in Lakshadweep Islands
482,Sao Miguel Do Iguacu,AU,-35.12,139.27,2020-02-11 01:13:03,79.00,86,0,5.01,Balcony On Sixth Lodge


In [11]:
# Take lat/lng for hotel, plot using Gmaps
# Ten or fewer pins for all the hotels/cities in the dataFrame
# Pins are clickable to display City, Country and Hotel Name
# Place on top of the heatmap.

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

In [12]:
hotel_info = [info_box_template.format(**row) for index, row in destinations.iterrows()]
selects_locations = destinations[["Latitude", "Longitude"]]
selects_locations

,Latitude,Longitude
8,5.95,80.54
42,-15.20,12.15
85,5.97,80.42
129,6.12,81.12
149,-12.79,45.27
430,10.57,72.64
482,-35.12,139.27


In [17]:
hotels_layer = gmaps.marker_layer(
    selects_locations, 
    info_box_content=hotel_info
)

fig.add_layer(hotels_layer)
fig

Figure(layout=FigureLayout(height='420px'))